2025-11-24 14:27:41 Monday first create

### 1.为什么llm领域，要分成训练和推理框架？
本质原因是对于llm，训练方式和推理方式是两种完全不同的计算过程，且有不同的核心目标。

- 对于训练阶段，
    - 过程：是一次性输入完整句子，计算整个句子的loss，反向传播；
    - 核心目标： 如何让大 Batch 更快地并行计算（最大化吞吐量）。
- 对于推理阶段，
    - 过程：首先输入完整 prompt 得到初始输出，然后每一步将“已有上下文 + 新生成的 token”作为新的输入，反复生成下一个 token，直至完成回答。
    - 核心目标：让单个请求尽可能快地产生下一个/首个 token（低延迟）。


###  2.1 KV cache 是怎么对 LLM 推理做优化的？

KV cache 通过缓存历史 token 的 Key/Value，使得自回归推理中每一步只需计算当前新 token 的 Q/K/V（尺寸仅为 1×d），而不必重新计算整个历史序列的 K/V。
这样注意力计算从需要对整条序列重算一次（整体复杂度约为 **O(n²)**），变成只用当前 Q 与缓存的 K/V 做一次查询（复杂度 **O(n)**）。

这避免了对历史部分的重复计算，从而显著加速每一步 token 的生成。





### 2.2基于KV-cache的注意力计算过程（以单头情况为例）

**1. 第 (i) 个 token 的输入向量表示当前要处理的状态。**
$x_i \in \mathbb{R}^{d}$

**2. 历史序列的 Key/Value 已经存入缓存，用于后续注意力查询。**
$K_{\text{cache}} \in \mathbb{R}^{(i-1)\times d},\quad V_{\text{cache}} \in \mathbb{R}^{(i-1)\times d}$

**3. 当前 token 经过线性变换得到查询、键和值的向量。**
$Q_i = x_i W_Q \in \mathbb{R}^{d},\quad K_i = x_i W_K \in \mathbb{R}^{d},\quad V_i = x_i W_V \in \mathbb{R}^{d}$

**4. 新计算得到的 (K_i, V_i) 会被追加到缓存，形成完整的历史序列。**
$K = \begin{bmatrix} K_{\text{cache}} \\ K_i \end{bmatrix} \in \mathbb{R}^{i\times d},\quad
V = \begin{bmatrix} V_{\text{cache}} \\ V_i \end{bmatrix} \in \mathbb{R}^{i\times d}$

**5. 当前查询向量与所有历史键相量做矩阵乘法，得到注意力得分向量。**
<!-- $\alpha_{i,j} = \dfrac{Q_i \cdot K_j}{\sqrt{d}},\quad \alpha_{i,j}\in\mathbb{R},j=1,\dots,i$ -->

$$ A_i = \dfrac{Q_i  K^T}{\sqrt{d}},\quad A_i \in\mathbb{R}^{1\times i}$$

**6. 经 softmax 归一化后得到注意力权重，用于决定当前 token 如何聚合历史信息。**

$$\omega_i = \text{softmax}(A_i),\quad \omega_i \in \mathbb{R}^{1 \times i}$$


**7. 最终输出是所有历史 Value 的加权求和，表示对当前 token 的注意力汇聚结果。**
<!-- $y_i = \sum_{j=1}^{i}\omega_{i,j} V_j,\quad y_i\in\mathbb{R}^{d}$ -->

$$y_i = \omega_i V ,\quad y_i \in \mathbb{R}^{1 \times d}$$




### 2.3 没有 KV-cache 的朴素自回归推理（以单头情况为例）

**1. 第 (i) 个 token 的输入序列由所有已生成的 token 拼接得到。**
$$
X =
\begin{bmatrix}
x_1\\
x_2\\
\vdots\\
x_i
\end{bmatrix}
\in \mathbb{R}^{i\times d}
$$

**2. 对整个序列中的所有 token 重新计算 Query / Key / Value（无缓存时每一步都要重算）。**
$$
Q = X W_Q \in \mathbb{R}^{i\times d},\qquad
K = X W_K \in \mathbb{R}^{i\times d},\qquad
V = X W_V \in \mathbb{R}^{i\times d}
$$

**3. 取出第 (i) 个位置的查询向量，用于计算当前 token 的注意力。**
$$
Q_i = Q[i] \in \mathbb{R}^{d}
$$

**4. 将当前查询向量与所有键向量做矩阵乘法，得到注意力得分向量。**
$$
A_i = \frac{Q_i K^\top}{\sqrt{d}}
\in \mathbb{R}^{1\times i}
$$

**5. 对注意力得分做 softmax，得到注意力权重向量。**
$$
\omega_i = \mathrm{softmax}(A_i)
\in \mathbb{R}^{1\times i}
$$

**6. 以注意力权重对所有 Value 做加权和，得到当前 token 的输出。**
$$
y_i = \omega_i V
\in \mathbb{R}^{1\times d}
$$




### 3.1. 常见的llm推理框架

| 框架名称                                           | 开发团队                | 开源时间 | 特点（主要实现语言 & 适用场景 & 支撑文献）                                                                                                                                                                    |
| ---------------------------------------------- | ------------------- | ---- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **vLLM**                                       | UC Berkeley + LMSys | 2023 | **语言：Python + CUDA**。核心技术 **PagedAttention**（分页 KV Cache），多请求并行推理吞吐最高。常用于 GPT/LLaMA/Qwen 部署。**支撑文献：《Efficient Memory Management for LLM Inference: PagedAttention》2023（官方论文）。GitHub vLLM。** |
| **SGLang**                                     | Stanford + LMSys    | 2024 | **语言：Python + CUDA**。支持 spec decoding、chat-level 优化，比 vLLM 更适合多轮对话推理。**支撑来源：SGLang 官方论文《SGLang: Efficient Communication-Aware LLM Serving》2024。**                                           |
| **TensorRT-LLM**                               | NVIDIA              | 2023 | **语言：C++/CUDA**。kernel fusion、FP8/INT8 量化、spec decode，可实现最低延迟推理。适用于 GPU 生产部署。**支撑文献：TensorRT-LLM 技术报告；NVIDIA Developer Blog 2023。**                                                         |
| **HuggingFace TGI（Text Generation Inference）** | HuggingFace         | 2022 | **语言：Rust + Python**。企业级 LLM 推理服务框架，支持 batching、streaming、LoRA merging。**支撑来源：HuggingFace 官方文档、TGI GitHub。**                                                                                |
| **llama.cpp / ggml / gguf**                    | Georgi Gerganov     | 2023 | **语言：C/C++ SIMD**。适用于 CPU / 移动端 / 边缘设备的 4bit/5bit/8bit 量化推理，适配 LLaMA/Gemma/Qwen 等。**支撑文献：llama.cpp README；ggml 设计文档。**                                                                      |
| **DeepSpeed-Inference**                        | Microsoft           | 2020 | **语言：Python + CUDA**。DeepSpeed 的推理子模块，kernel 合并、INT8 量化，适合训练+推理一体化的团队。**支撑来源：DeepSpeed inference 文档；微软技术报告。**                                                                               |
| **LightLLM（字节）**                               | ByteDance           | 2023 | **语言：Python + Triton**。面向推理的轻量化引擎，支持动态 KV 管理、少量显存高效推理。**支撑来源：字节 LightLLM 文档；GitHub。**                                                                                                       |
| **MindSpore-LLM（华为）**                          | Huawei              | 2023 | **语言：Python / Ascend CANN**。昇腾平台优化推理（静态图 + 重编译）。适合国产部署。**支撑来源：MindSpore-LLM 文档。**                                                                                                           |


### 3.2 常见的llm训练框架

| 框架名称                                          | 开发团队               | 开源时间                   | 特点（主要实现语言 & 适用场景 & 支撑文献）                                                                                                                                                                                             |
| --------------------------------------------- | ------------------ | ---------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **DeepSpeed**                                 | Microsoft          | 2020                   | **语言：Python + CUDA**。适用于超大模型训练（GPT-3/OPT/GLM）。核心技术为 **ZeRO**（Zero Redundancy Optimizer）→ 将模型参数、梯度、优化器完全分片。**支撑文献：DeepSpeed 论文《ZeRO: Memory Optimizations Toward Training Trillion Parameter Models》2020。官方 GitHub。** |
| **Megatron-LM**                               | NVIDIA             | 2019                   | **语言：Python + CUDA**。适用于 GPT/GPT-NeoX/LLaMA 等大模型的张量并行（Tensor Parallel）与流水线并行（Pipeline Parallel）。**支撑文献：NVIDIA 官方论文《Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism》2019。**     |
| **Colossal-AI**                               | HPC-AI Tech（梁上燕团队） | 2022                   | **语言：Python + CUDA**。专注低成本大模型训练（hybrid parallel），支持 GPT/PaLM/ViT。**支撑文献：Colossal-AI 白皮书《Colossal-AI: A Unified Deep Learning System For Large-Scale Parallel Training》2022。**                                        |
| **PyTorch FSDP（Fully Sharded Data Parallel）** | Meta               | 2021                   | **语言：C++/Python**。官方原生分布式训练框架，适用大模型参数完全分片训练，显存效率高。**支撑文献：Meta 官方文档《PyTorch FSDP Design Note》；论文《Fully Sharded Data Parallel》PyTorch 2021。**                                                                          |
| **JAX + Flax**                                | Google             | JAX(2018) / Flax(2019) | **语言：Python + XLA 编译器**。用于 PaLM、Gemini 的训练生态，自动向量化（vmap,pmap）、加速 TPU/GPU 大规模训练。**支撑文献：Google JAX 论文《JAX:Composable Transformations of Python+NumPy Programs》2018；PaLM 论文（2022）。**                                    |
| **MindFormers（华为）**                           | Huawei             | 2022                   | **语言：Python / Ascend CANN**。昇腾（Ascend）芯片生态下的大模型训练框架，整合 DP/TP/PP。适合国产 LLM 训练。**支撑文献：华为 MindFormers 文档；MindSpore 白皮书。**                                                                                                |
